# Titanic

#### Instructions d'exécution : 
Le fichier csv appelé 'Titanic_data.csv' et celui ci 'tp_titanic.ipynb' doivent être stockés dans un même dossier.

#### Préambule
En 1912, le Titanic a coulé après avoir percuté un iceberg. \
\
Nous disposons d'un fichier de données concernant les passagers et aimerions rendre compte du type de passagers présents sur le bateau et des facteurs de survie des passagers du bateau en fonction de différents critères tels que l'âge, le sexe, la classe sociale ou encore le prix du ticket afin de déterminer, s'il y en a, les facteurs ayant pu favoriser la survie de certaines catégories de passagers.\
\
Hypothèses : \
\
.    On suppose que les enfants auraient été sauvés en majorité, suivis par les femmes au détriment des hommes.\
\
.    On suppose également que la classe sociale a pu jouer un rôle dans le destin des passagers.\
\
.    On peut également imaginer qu'au sein d'une même classe sociale, une différence de prix d'achat du ticket existe entre les survivants et les victimes. 

#### Création de la sparksession et importation des fonctions nécessaires pour la suite : 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, ceil, avg, regexp_replace, isnull

from pyspark.sql.types import IntegerType,BooleanType,DateType

spark = SparkSession.builder.master("local[*]").appName("TP à rendre").getOrCreate()

spark

## 1. Importation et lecture du fichier de données concernant les passagers du Titanic en tant que DataFrame et en tant que RDD (utilisation dans le point 10) et description de son contenu.

In [2]:
df_titanic = spark.read.csv('./titanic_data.csv', header = True, sep = ';')
rdd = spark.sparkContext.textFile("./Titanic_data.csv")
df_titanic.show()

+------+--------+--------------------+------+------+-----+-----+--------+--------+-------+--------+----+----+--------------------+
|pclass|survived|                name|   sex|   age|sibsp|parch|  ticket|    fare|  cabin|embarked|boat|body|           home.dest|
+------+--------+--------------------+------+------+-----+-----+--------+--------+-------+--------+----+----+--------------------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|   24160|211,3375|     B5|       S|   2|NULL|        St Louis, MO|
|     1|       1|Allison, Master. ...|  male|0,9167|    1|    2|  113781|151,5500|C22 C26|       S|  11|NULL|Montreal, PQ / Ch...|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|  113781|151,5500|C22 C26|       S|NULL|NULL|Montreal, PQ / Ch...|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|  113781|151,5500|C22 C26|       S|NULL| 135|Montreal, PQ / Ch...|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|  113781|151,5500|C

#### Affichage du contenu de chaque colonne


In [3]:
df_titanic.columns

columns_convert = {'pclass': 'The ticket class of the passenger, which can be 1st (1), 2nd (2), or 3rd (3) class.',
'survived' : 'A binary variable indicating whether the passenger survived (1) or did not survive (0).',
'name' : 'The name of the passenger',
'sex' : 'The gender of the passenger (male or female)',
'age' : 'The age of the passenger in years, it may contain missing values',
'sibsp' : 'The number of siblings or spouses the passenger had aboard the Titanic',
'parch' : 'The number of parents or children the passenger had aboard the Titanic.',
'ticket' : 'The ticket number',
'fare' : 'The amount of money the passenger paid for the ticket.',
'cabin' : 'The cabin number where the passenger stayed. It may contain missing values',
'embarked' : 'The port at which the passenger boarded the Titanic (C for Cherbourg, Q for Queenstown, S for Southampton',
'boat' : 'The boat which rescued the passenger.',
'body' : 'Id of the body',
'home.dest' : 'Where the passenger wanted to stay after his trip.'}
for key, value in columns_convert.items() : 
    print(key, ' : ', value)


pclass  :  The ticket class of the passenger, which can be 1st (1), 2nd (2), or 3rd (3) class.
survived  :  A binary variable indicating whether the passenger survived (1) or did not survive (0).
name  :  The name of the passenger
sex  :  The gender of the passenger (male or female)
age  :  The age of the passenger in years, it may contain missing values
sibsp  :  The number of siblings or spouses the passenger had aboard the Titanic
parch  :  The number of parents or children the passenger had aboard the Titanic.
ticket  :  The ticket number
fare  :  The amount of money the passenger paid for the ticket.
cabin  :  The cabin number where the passenger stayed. It may contain missing values
embarked  :  The port at which the passenger boarded the Titanic (C for Cherbourg, Q for Queenstown, S for Southampton
boat  :  The boat which rescued the passenger.
body  :  Id of the body
home.dest  :  Where the passenger wanted to stay after his trip.


## 2. Nettoyage des données
Les passagers dont l'âge n'est pas indiqué risquent de perturber les prochains résultats donc on préfère les retirer pour les calculs à suivre.

In [4]:
df_titanic = df_titanic.na.drop(subset=['age'])

## 3. Statistiques générales sur les passagers : 

In [5]:
print('Sur le nombre total de passagers, on constate : ',)
print("     Pourcentage total de femmes : ", round(df_titanic.filter((df_titanic['sex'] == 'female') & (df_titanic['age'] >= '18')).count() *100 / df_titanic.count(),2), "%.")
print("     Pourcentage total d'hommes : ",round(df_titanic.filter((df_titanic['sex'] == 'male') & (df_titanic['age'] >= '18')).count() *100 / df_titanic.count(),2), "%.")
print("     Pourcentage total d'enfants : ",round(df_titanic.filter('age < 18').count() *100 / df_titanic.count(),2), "%.")
print('     Nombre de personnes survivantes : ',df_titanic.filter('survived == 1').count(),' sur un total de ',df_titanic.count(), ' passagers')
print('     Nombre de victimes : ', df_titanic.filter('survived == 0').count(),' sur un total de ',df_titanic.count(), ' passagers')
print('     Taux de mortalité dû à la catastrophe : ', round(df_titanic.filter('survived == 0').count()*100 / df_titanic.count(),2), '%.')
print('     Taux de survie à la catastrophe : ', round(df_titanic.filter('survived == 1').count()*100 / df_titanic.count(),2), '%.')

Sur le nombre total de passagers, on constate : 
     Pourcentage total de femmes :  33.08 %.
     Pourcentage total d'hommes :  57.93 %.
     Pourcentage total d'enfants :  13.29 %.
     Nombre de personnes survivantes :  427  sur un total de  1046  passagers
     Nombre de victimes :  619  sur un total de  1046  passagers
     Taux de mortalité dû à la catastrophe :  59.18 %.
     Taux de survie à la catastrophe :  40.82 %.


## 4. Pour les comparaisons à venir, on caste les colonnes age et fare de str en float dans les colonnes age_int et fare_int

In [6]:
df_titanic = df_titanic.withColumn('age', regexp_replace('age',',','.') )
df_titanic = df_titanic.withColumn('age_int', df_titanic['age'].cast('float'))

df_titanic = df_titanic.withColumn('fare', regexp_replace('fare',',','.') )
df_titanic = df_titanic.withColumn('fare_int', df_titanic['fare'].cast('float'))


## 5. Moyennes d'âge en fonction des différentes catégories (général, hommes, femmes, enfants) chez les survivants et les victimes

In [7]:
df_titanic.select(avg(col("age_int")).alias('Age moyen des passagers')).show()
df_titanic.filter(df_titanic['survived'] =='1').select(avg(col("age_int")).alias('Age moyen des survivants')).show()
df_titanic.filter(df_titanic['survived'] =='0').select(avg(col("age_int")).alias('Age moyen des victimes')).show()

df_titanic.filter(df_titanic['age_int'] <18).select(avg(col("age_int")).alias('Age moyen des enfants')).show()
df_titanic.filter((df_titanic['age_int'] <18) & (df_titanic['survived'] == '1')).select(avg(col("age_int")).alias('Age moyen des enfants survivants')).show()
df_titanic.filter((df_titanic['age_int'] <18) & (df_titanic['survived'] == '0')).select(avg(col("age_int")).alias('Age moyen des enfants décédés')).show()

df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'male')).select(avg(col("age_int")).alias('Age moyen des hommes')).show()
df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'male') & (df_titanic['survived'] == '1')).select(avg(col("age_int")).alias('Age moyen des hommes survivants')).show()
df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'male') & (df_titanic['survived'] == '0')).select(avg(col("age_int")).alias('Age moyen des hommes décédés')).show()

df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'female')).select(avg(col("age_int")).alias('Age moyen des femmes')).show()
df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'female') & (df_titanic['survived'] == '1')).select(avg(col("age_int")).alias('Age moyen des femmes survivantes')).show()
df_titanic.filter((df_titanic['age_int'] >=18) & (df_titanic['sex'] == 'female') & (df_titanic['survived'] == '0')).select(avg(col("age_int")).alias('Age moyen des femmes décédées')).show()

+-----------------------+
|Age moyen des passagers|
+-----------------------+
|     29.881134512433587|
+-----------------------+

+------------------------+
|Age moyen des survivants|
+------------------------+
|      28.918228103070405|
+------------------------+

+----------------------+
|Age moyen des victimes|
+----------------------+
|    30.545368820669577|
+----------------------+

+---------------------+
|Age moyen des enfants|
+---------------------+
|    9.101731818217736|
+---------------------+

+--------------------------------+
|Age moyen des enfants survivants|
+--------------------------------+
|               8.013375308778551|
+--------------------------------+

+-----------------------------+
|Age moyen des enfants décédés|
+-----------------------------+
|           10.309360273896832|
+-----------------------------+

+--------------------+
|Age moyen des hommes|
+--------------------+
|          33.6328125|
+--------------------+

+-------------------------------+

## 6. Taux de mortalité chez les adultes, en fonction du sexe des adultes et chez les enfants 

In [8]:
print(round(df_titanic.filter((df_titanic['age_int'] < '18') & (df_titanic['survived'] =='0')).count()*100 / df_titanic.filter((df_titanic['age_int'] < '18') ).count(),2), "% des enfants sont décédés dans la catastrophe.")

print(round(df_titanic.filter((df_titanic['age_int'] >= '18') & (df_titanic['survived'] =='0')).count()*100 / df_titanic.filter((df_titanic['age_int'] >= '18') ).count(),2), "% des adultes sont décédés dans la catastrophe.")

print(round(df_titanic.filter((df_titanic['age_int'] >= '18') & (df_titanic['survived'] =='0') & (df_titanic['sex'] == 'female')).count()*100 / df_titanic.filter((df_titanic['age_int'] >= '18') &  (df_titanic['sex'] == 'female')).count(),2), "% des femmes sont décédées dans la catastrophe.")

print(round(df_titanic.filter((df_titanic['age_int'] >= '18') & (df_titanic['survived'] =='0') & (df_titanic['sex'] == 'male')).count()*100 / df_titanic.filter((df_titanic['age_int'] >= '18') &  (df_titanic['sex'] == 'male')).count(),2), "% des hommes sont décédés dans la catastrophe.")

print("")

print('Parmi les survivants, on constate : ',df_titanic.filter((df_titanic['age_int'] >= '18') & (df_titanic['survived'] == '1') & (df_titanic['sex'] == 'female')).count(), ' femmes et ',df_titanic.filter((df_titanic['age_int'] >= '18') & (df_titanic['survived'] =='1') & (df_titanic['sex'] == 'male')).count(), ' hommes et ',df_titanic.filter((df_titanic['age_int'] < '18') & (df_titanic['survived'] =='1')).count(), 'enfants.' )

print('     Soit un ratio de ', round(df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] == '1') & (df_titanic['sex'] == 'female')).count()*100 / df_titanic.filter('survived ==1').count(),2), '% de femmes parmi les survivants, ', round(df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] =='1') & (df_titanic['sex'] == 'male')).count()*100 / df_titanic.filter('survived ==1').count(),2),"% d'hommes et", round(df_titanic.filter((df_titanic['age'] < '18') & (df_titanic['survived'] =='1')).count()*100 / df_titanic.filter('survived ==1').count(),2) , "% d'enfants.")

print("")

print('Parmi les victimes, on constate : ',df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] == '0') & (df_titanic['sex'] == 'female')).count(), ' femmes et ',df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] =='0') & (df_titanic['sex'] == 'male')).count(), ' hommes et ',df_titanic.filter((df_titanic['age'] < '18') & (df_titanic['survived'] =='0')).count(), 'enfants.' )

print('     Soit un ratio de ', round(df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] == '0') & (df_titanic['sex'] == 'female')).count()*100 / df_titanic.filter('survived ==0').count(),2), '% de femmes parmi les victimes, ', round(df_titanic.filter((df_titanic['age'] >= '18') & (df_titanic['survived'] =='0') & (df_titanic['sex'] == 'male')).count()*100 / df_titanic.filter('survived ==0').count(),2),"% d'hommes et", round(df_titanic.filter((df_titanic['age'] < '18') & (df_titanic['survived'] =='0')).count()*100 / df_titanic.filter('survived ==0').count(),2) , "% d'enfants.")



47.4 % des enfants sont décédés dans la catastrophe.
61.21 % des adultes sont décédés dans la catastrophe.
23.42 % des femmes sont décédées dans la catastrophe.
81.94 % des hommes sont décédés dans la catastrophe.

Parmi les survivants, on constate :  242  femmes et  104  hommes et  81 enfants.
     Soit un ratio de  60.66 % de femmes parmi les survivants,  28.1 % d'hommes et 11.24 % d'enfants.

Parmi les victimes, on constate :  87  femmes et  486  hommes et  46 enfants.
     Soit un ratio de  14.05 % de femmes parmi les victimes,  78.51 % d'hommes et 7.43 % d'enfants.


## 7. Calcul du prix moyen payé par ticket en fonction de la classe sociale

In [9]:
df_titanic.filter(df_titanic['pclass'] == '1').select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket première classe.')).show()

+--------------------------------------------------+
|Montant moyen payé pour un ticket première classe.|
+--------------------------------------------------+
|                                 92.22935866637968|
+--------------------------------------------------+



In [10]:
df_titanic.filter(df_titanic['pclass'] == '2').select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket seconde classe.')).show()

+-------------------------------------------------+
|Montant moyen payé pour un ticket seconde classe.|
+-------------------------------------------------+
|                               21.855044474546936|
+-------------------------------------------------+



In [11]:
df_titanic.filter((df_titanic['pclass'] == '3')).select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket troisième classe.')).show()

+---------------------------------------------------+
|Montant moyen payé pour un ticket troisième classe.|
+---------------------------------------------------+
|                                 12.879299092769623|
+---------------------------------------------------+



## 8. Comparaison du prix moyen payé en général et pour chaque classe chez les survivants et chez les victimes

#### Première classe

##### Prix moyen première classe

In [12]:
df_titanic.filter(df_titanic['pclass'] == '1').select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket première classe.')).show()

+--------------------------------------------------+
|Montant moyen payé pour un ticket première classe.|
+--------------------------------------------------+
|                                 92.22935866637968|
+--------------------------------------------------+



##### Prix moyen première classe pour les survivants

In [13]:
df_titanic.filter((df_titanic['pclass'] == '1') & (df_titanic['survived'] == '1') ).select(avg(col("fare_int")).alias('Montant moyen payé par les survivants pour un ticket première classe.')).show()

+---------------------------------------------------------------------+
|Montant moyen payé par les survivants pour un ticket première classe.|
+---------------------------------------------------------------------+
|                                                   102.21699151940109|
+---------------------------------------------------------------------+



##### Prix moyen première classe pour les victimes

In [14]:
df_titanic.filter((df_titanic['pclass'] == '1') & (df_titanic['survived'] == '0') ).select(avg(col("fare_int")).alias('Montant moyen payé par les survivants pour un ticket première classe.')).show()

+---------------------------------------------------------------------+
|Montant moyen payé par les survivants pour un ticket première classe.|
+---------------------------------------------------------------------+
|                                                    74.67827569165276|
+---------------------------------------------------------------------+



#### Seconde classe

##### Prix moyen seconde classe

In [15]:
df_titanic.filter(df_titanic['pclass'] == '2').select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket seconde classe.')).show()

+-------------------------------------------------+
|Montant moyen payé pour un ticket seconde classe.|
+-------------------------------------------------+
|                               21.855044474546936|
+-------------------------------------------------+



##### Prix moyen seconde classe pour les survivants

In [16]:
df_titanic.filter((df_titanic['pclass'] == '2') & (df_titanic['survived'] == '1')).select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket seconde classe.')).show()

+-------------------------------------------------+
|Montant moyen payé pour un ticket seconde classe.|
+-------------------------------------------------+
|                               23.180471345652705|
+-------------------------------------------------+



##### Prix moyen seconde classe pour les victimes

In [17]:
df_titanic.filter((df_titanic['pclass'] == '2') & (df_titanic['survived'] == '0')).select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket seconde classe.')).show()

+-------------------------------------------------+
|Montant moyen payé pour un ticket seconde classe.|
+-------------------------------------------------+
|                               20.811043856895132|
+-------------------------------------------------+



#### Troisième classe

##### Prix moyen troisième classe

In [18]:
df_titanic.filter(df_titanic['pclass'] == '3').select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket troisième classe.')).show()

+---------------------------------------------------+
|Montant moyen payé pour un ticket troisième classe.|
+---------------------------------------------------+
|                                 12.879299092769623|
+---------------------------------------------------+



##### Prix moyen troisième classe pour les survivants

In [19]:
df_titanic.filter((df_titanic['pclass'] == '3') & (df_titanic['survived'] == '1')).select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket troisième classe.')).show()

+---------------------------------------------------+
|Montant moyen payé pour un ticket troisième classe.|
+---------------------------------------------------+
|                                 12.427448958840989|
+---------------------------------------------------+



##### Prix moyen troisième classe pour les victimes

In [20]:
df_titanic.filter((df_titanic['pclass'] == '3') & (df_titanic['survived'] == '0')).select(avg(col("fare_int")).alias('Montant moyen payé pour un ticket troisième classe.')).show()

+---------------------------------------------------+
|Montant moyen payé pour un ticket troisième classe.|
+---------------------------------------------------+
|                                 13.039712012944829|
+---------------------------------------------------+



## 9. Taux de survie en fonction de la classe sociale


#### Taux de survie des première classe

In [21]:
print('Taux de survie des passagers de première classe : ', round(df_titanic.filter((df_titanic['pclass'] == 1) & (df_titanic['survived'] == 1)).count() *100 / df_titanic.filter(df_titanic['pclass'] == 1).count(),2), "%.")

Taux de survie des passagers de première classe :  63.73 %.


#### Taux de survie des seconde classe

In [22]:
print('Taux de survie des passagers de seconde classe : ', round(df_titanic.filter((df_titanic['pclass'] == 2) & (df_titanic['survived'] == 1)).count() *100 / df_titanic.filter(df_titanic['pclass'] == 2).count(),2), "%.")

Taux de survie des passagers de seconde classe :  44.06 %.


#### Taux de survie des troisième classe

In [23]:
print('Taux de survie des passagers de troisième classe : ', round(df_titanic.filter((df_titanic['pclass'] == 3) & (df_titanic['survived'] == 1)).count() *100 / df_titanic.filter(df_titanic['pclass'] == 3).count(),2), "%.")

Taux de survie des passagers de troisième classe :  26.15 %.


In [24]:
# Pourcentage de survivants ayant embarqué sur un canot 
print((df_titanic.filter(df_titanic['survived'] == 1) (df_titanic['boat'])))
      #*100 / df_titanic.filter(df_titanic['survived'] == 1).count(),2))

# Nombre de personnes par canot (GroupBy)




TypeError: 'DataFrame' object is not callable

## 10. Retour sur le nettoyage des données et la relativité des résultats induite par l'absence de certaines d'entre elles

In [25]:
print("Nombre de passagers initial : ",rdd.count())
print("Nombre de passagers pris en compte dans cette étude : ",df_titanic.count())
print("Nombre de passagers non pris en compte en raison d'absence d'informations sur leur âge : ",(rdd.count()) - (df_titanic.count()), "\n Soit : ", round((rdd.count() - df_titanic.count())*100/(rdd.count()),2), "% des passagers présents à bord non pris en compte.") 

Nombre de passagers initial :  1311
Nombre de passagers pris en compte dans cette étude :  1046
Nombre de passagers non pris en compte en raison d'absence d'informations sur leur âge :  265 
 Soit :  20.21 % des passagers présents à bord non pris en compte.


# Conclusions

On constate un taux de survie général de 40,82%.\
On ne note pas de différence d'âge entre les survivants et les victimes de manière générale, cependant les enfants survivants ont autour de 8 ans tandis que les enfants décédés ont autour de 10 ans, donc plus âgés. \
Les femmes ayant survécu ont en moyenne 34 ans tandis que celles ayant péri ont en moyenne 30 ans donc plus jeunes.\
Les hommes, quant à eux, survivants comme victimes, ont en moyenne 33 ans.\
\
La moitié des enfants ont été sauvés.\
3/4 à 4/5èmes des femmes ont survécu.\
En revanche, seuls 20% des hommes ont été sauvés.\
\
En ce qui concerne le prix des tickets : \
Le ticket de première classe coûtait en moyenne 92,22 £. Les survivants l'ayant en moyenne payé plus cher (102 £) que les victimes (74 £).\
En seconde classe, il coûtait en moyenne 21,85 £ ; 23 £ chez les survivants contre 20 £ chez les victimes.\
Enfin, en troisième classe, le ticket a été payé 12,88 £ en moyenne ; 12,42 £ chez les survivants et 13 £ chez les victimes.\
\
On note que le taux de survie varie énormément en fonction de la classe sociale.\
63.73% chez les première classe, 44.06% chez les seconde classe et seulement 26.15% chez les première classe.

##### On supposait que les enfants auraient été sauvés en majorité, suivis par les femmes au détriment des hommes.
Ce n'est pas le cas, ce sont les femmes qui ont été sauvées en majorité, puis les enfants. Cependant, cela s'est effectivement fait au détriment des hommes car 80% d'entre eux sont décédés dans le naufrage.

##### On supposait également que la classe sociale ait pu jouer un rôle dans le destin des passagers.
C'est le cas, on a remarqué qu'un taux de survie élevé était nettement corrélé à une classe sociale haute.  

##### On supposait qu'au sein d'une même classe sociale, une différence de prix d'achat du ticket existait entre les survivants et les victimes. 
Effectivement, c'est le cas, mais surtout chez les première classe, ça ne l'est pas chez les troisième classe et très peu chez les seconde classe.







Cette étude est à relativiser car sur le jeu de données utilisé, 20% des passagers réellement présents n'ont pas été pris en compte car leur âge n'était pas donné pour plus de praticité.